In [51]:
import fingertips_py as ftp
import pandas as pd
import requests

I need to know for each indicator, which area types and year conifgurations exist so that I know which ones I can pick from and minimise computation for data extraction. - Need to see if I can get a list of this

The alternative would be to create a hierarchy of preference and iterate through the data for each hierachy level. If an indicator is found within a level, then it will no longer be queried for further levels. This might still be computationally expensive but needs to be investigated further 

# Area types available for each indicator

In [52]:
raw_available_indicator_at_area = pd.read_json('https://fingertips.phe.org.uk/api/available_data').fillna(value='null')

print(raw_available_indicator_at_area['AreaTypeId'].unique()
      ,len(raw_available_indicator_at_area['AreaTypeId'].unique()),
      len(raw_available_indicator_at_area),
      raw_available_indicator_at_area['IndicatorId'].nunique()
    #   ,
    #   available_indicator_at_area['IndicatorId'].unique()
)

[402 401 102 101  15   6 221 220 204 167 166  66   7 302 219 152 301 223
 202 165 154 201  46  10 126 104 119 203 118 205 112 129 222   8   3  41] 36 8947 1698


In [53]:
raw_available_indicator_at_area.columns

Index(['IndicatorId', 'AreaTypeId'], dtype='object')

# Indicator metadata
To get the name of each indicator

In [54]:
indicator_metadata = pd.read_csv('https://fingertips.phe.org.uk/api/indicator_metadata/csv/all')

In [55]:
indicator_metadata['Indicator ID'].nunique()

1408

# Area Ids metadata
To get the name of the area

In [56]:
area_ref = pd.read_json('https://fingertips.phe.org.uk/api/area_types').fillna(value='null')

In [57]:
area_ref.columns

Index(['Id', 'Name', 'Short', 'Class', 'Sequence', 'CanBeDisplayedOnMap'], dtype='object')

## hierarchy

In [58]:
area_hierarchy = pd.read_csv('https://docs.google.com/spreadsheets/u/0/d/15RhWWsHPPMLWoxR5sJcpK-vraRkidRY8jsAb_Y_5GwI/gviz/tq?tqx=out:csv&tq&gid=963757659&headers=1')
area_hierarchy = area_hierarchy[['AreaTypeId', 'map_position', 'hierarchy']].dropna()

# Area types available for each indicator with extra metadata

## prepare full table

In [60]:
available_indicator_at_area = pd.merge(left=raw_available_indicator_at_area, right=area_ref,
         how='left', left_on='AreaTypeId', right_on='Id',
         indicator=True).rename({'_merge': 'area_merge'}, axis=1)

available_indicator_at_area = pd.merge(left=available_indicator_at_area, right=indicator_metadata[['Indicator ID', 'Indicator']],
                                       left_on='IndicatorId', right_on='Indicator ID', how='inner',
                                       indicator=True, )

## create views

In [62]:
# indicators_per_area = (pd.DataFrame(available_indicator_at_area.groupby(['Name', 'Short', 'Class', 'Sequence','CanBeDisplayedOnMap'])['_merge'].count())
#                        .reset_index()
#                        .sort_values(['Class', 'Sequence'], ascending=False)
#                        .rename({'_merge': 'indicator_count'}, axis=1)
#                        )

In [63]:
# indicator_at_area_pivot = available_indicator_at_area[['Indicator', 'Name', 'Id']].pivot_table(index='Indicator', columns='Name', aggfunc='count')

## use hierarchy to choose indicators and areas

In [73]:
full_area_indicator_hierarchy = pd.merge(left=area_hierarchy, right=raw_available_indicator_at_area,
         left_on='AreaTypeId', right_on='AreaTypeId',
         how='inner')

In [79]:
left = full_area_indicator_hierarchy[full_area_indicator_hierarchy['map_position']=='left']
left['rank'] = left.groupby('IndicatorId')['hierarchy'].rank(method='dense')
keep_left = left[left['rank']==1]


C:\Users\medssam\AppData\Local\Temp\ipykernel_14844\2219602755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  left['rank'] = left.groupby('IndicatorId')['hierarchy'].rank(method='dense')


In [81]:
keep_left['rank'].unique()

array([1.])

In [111]:
def find_first_hierarchy(data, map_position):
    data = data[data['map_position']==map_position].loc[:, ['IndicatorId','hierarchy','AreaTypeId', 'map_position']]
    data['rank'] = data.groupby('IndicatorId')['hierarchy'].rank(method='dense')
    return data[data['rank']==1].drop('rank', axis=1)

In [113]:
get_indicator_area = pd.concat([find_first_hierarchy(full_area_indicator_hierarchy, 'left'),
                                find_first_hierarchy(full_area_indicator_hierarchy, 'middle'),
                                find_first_hierarchy(full_area_indicator_hierarchy, 'right')]
)

# Retrieve indicator data

## filter by those that only have all 3 as a test case

In [125]:
number_of_maps_returned = get_indicator_area.groupby('IndicatorId')['map_position'].count()
ind_ids_keep_3_maps = number_of_maps_returned[number_of_maps_returned==3].index.values

In [127]:
get_ind_area_combos = get_indicator_area[get_indicator_area['IndicatorId'].isin(ind_ids_keep_3_maps)]

In [128]:
get_ind_area_combos

,IndicatorId,hierarchy,AreaTypeId,map_position
1,200,1.0,221,left
2,212,1.0,221,left
3,219,1.0,221,left
5,241,1.0,221,left
6,247,1.0,221,left
...,...,...,...,...
6096,92263,8.0,102,right
6097,92264,8.0,102,right
6100,92272,8.0,102,right
6167,92866,8.0,102,right


In [65]:
# # metadata_areas = ftp.metadata.get_all_areas(is_test=False)
# # metadata_value_notes = ftp.metadata.get_all_value_notes(is_test=False)
# # metadata_profiles = ftp.metadata.get_all_profiles(is_test=False)
# # metadata_domains = ftp.metadata.get_domains_in_profile(18)
# metadata_all_indicators = ftp.metadata.get_metadata_for_all_indicators_from_csv(is_test=False)
# metadata_all_indicator_ids = metadata_all_indicators['Indicator ID'].tolist()

In [66]:
# metadata_all = pd.read_csv('https://fingertips.phe.org.uk/api/indicator_metadata/all?include_definition=yes&include_system_content=yes')

In [67]:
# replace_comma = '%2C'
# ids_as_str = replace_comma.join([str(i) for i in metadata_all_indicator_ids])

In [68]:
# data = ftp.retrieve_data.get_all_data_for_indicators(indicators=[108,
#  113,
#  114,
#  200,
#  212],
#                                                                area_type_id=7,
#                                                                parent_area_type_id=15,
#                                                                filter_by_area_codes=None,
#                                                                is_test=False)

In [69]:
# ids_as_str

In [70]:
# data = ftp.retrieve_data.get_data_by_indicator_ids(indicator_ids=ids_as_str, # [Maximum 100]
#                                                    area_type_id=7,
#                                                    parent_area_type_id=15,
#                                                    profile_id=None,
#                                                    include_sortable_time_periods=None,
#                                                    is_test=False)